Trying a keras without all the generators (loading all data in memory) to see if I then still get the same behavior.

In [1]:
from pathlib import Path
import tensorflow as tf

from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

In [2]:
path = Path("/home/jvlier/data/split-balanced/")

In [11]:
!ls $path

models	test  train  valid


In [12]:
import numpy as np

In [13]:
path / "train" / "BirdHome"

PosixPath('/home/jvlier/data/split-balanced/train/BirdHome')

In [14]:
IMG_X = 224
IMG_Y = 224

In [15]:
import cv2

In [16]:
def load_imgs(path):
    dest = None
    img_paths = list(path.glob("*.jpeg"))
    res = np.zeros((len(img_paths), IMG_Y, IMG_X, 3))
    for i, img_path in enumerate(img_paths):
        img = cv2.resize(plt.imread(img_path), (IMG_X, IMG_Y), cv2.INTER_CUBIC)
        res[i, :] = img
    return res
    

In [17]:
train_images_home = load_imgs(path / "train" / "BirdHome")
train_images_roam = load_imgs(path / "train" / "BirdRoaming")
train_labels = np.concatenate((np.zeros(len(train_images_home)), np.ones(len(train_images_roam))))
train_images = np.concatenate((train_images_home, train_images_roam)) / 255.0

In [10]:
test_images_home = load_imgs(path / "test" / "BirdHome")
test_images_roam = load_imgs(path / "test" / "BirdRoaming")
test_labels = np.concatenate((np.zeros(len(test_images_home)), np.ones(len(test_images_roam))))
test_images = np.concatenate((test_images_home, test_images_roam)) / 255.0

In [38]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_Y, IMG_X, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation="relu"))
model.add(layers.Dropout(.7))
model.add(layers.Dense(1, activation="sigmoid"))


In [39]:
model.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_15 (Conv2D)           (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 52, 52, 64)        36928     
_________________________________________________________________
flatten_5 (Flatten)          (None, 173056)            0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)               

In [41]:
from tensorflow.keras import optimizers

In [40]:
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=20, batch_size=8,
                    validation_data=(test_images, test_labels, )
)


Train on 1732 samples, validate on 578 samples
Epoch 1/20
1732/1732 [==============================] - 9s 5ms/sample - loss: 0.6771 - accuracy: 0.5953 - val_loss: 0.6560 - val_accuracy: 0.5796
Epoch 2/20
1732/1732 [==============================] - 8s 5ms/sample - loss: 0.5589 - accuracy: 0.7575 - val_loss: 0.6587 - val_accuracy: 0.5381
Epoch 3/20
1732/1732 [==============================] - 8s 5ms/sample - loss: 0.4502 - accuracy: 0.7927 - val_loss: 0.6812 - val_accuracy: 0.5433
Epoch 4/20
1732/1732 [==============================] - 8s 5ms/sample - loss: 0.3897 - accuracy: 0.8216 - val_loss: 0.7411 - val_accuracy: 0.5969
Epoch 5/20
1732/1732 [==============================] - 8s 5ms/sample - loss: 0.3609 - accuracy: 0.8222 - val_loss: 0.6983 - val_accuracy: 0.6315
Epoch 6/20
1732/1732 [==============================] - 8s 5ms/sample - loss: 0.3188 - accuracy: 0.8378 - val_loss: 0.6902 - val_accuracy: 0.6211
Epoch 7/20
1732/1732 [==============================] - 8s 5ms/sample - loss:

KeyboardInterrupt: 

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(train_images, train_labels, verbose=2, batch_size=16)


In [22]:
train_preds = model.predict_classes(train_images, batch_size=16)

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
accuracy_score(train_labels, train_preds)

0.9872979214780601

Ok, this seems better. But loading all data in memory isn't very scalable.
Maybe this is a good excuse to check out fast.ai.